In [1]:
import os, sys
import pandas as pd
import glob
import numpy as np

In [3]:
path = '../data/'
models =  ['Model'+str(i) for i in np.arange(1,17)]
body_parts = ['Shoulder', 'Humerus', 'Finger', 'Elbow', 'Wrist', 'Forearm', 'Hand']
filename = 'abnormal_results.csv'
color = {'Shoulder' : '#d73027', 
        'Humerus' : '#fc8d59', 
        'Finger' : '#fee090', 
        'Elbow' : '#ffffbf', 
        'Wrist' : '#e0f3f8', 
        'Forearm' : '#91bfdb', 
        'Hand' : '#4575b4', 
        'Input' : '#999999',
        'Normal' : '#5ab4ac',
        'Abnormal' : '#d8b365'}
data = {}

#read model results
for part in body_parts:
    for model in models:
        pathname = os.path.join(path, part, model, filename)
        name = part + '_' + model  
        data[name] = pd.read_csv(pathname)
        data[name].rename(columns = {'Abormal_Prediction' : 'Abnormal_Prediction'},
            inplace = True)


In [88]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
import pandas as pd
import math

class DenseNetViz():

    def __init__(self, kwargs):
        self.data = kwargs['data']
        self.body_parts = kwargs['body_parts']
        self.color = kwargs['color']
        self.nkeys = ['name', 'color', 'accuracy']
        self.lkeys = ['source', 'target', 'value', 'count', 'color']
        self.classes = ['Normal', 'Abnormal']
        self.ckeys = ['tn', 'fp', 'fn', 'tp']
        self.nodes = [] 
        self.links = []
        self.cm = {}

    def get(self, predPart, model = 0, view = 'all', truePart = 'none'):
        
        self.predPart = predPart
        self.truePart = truePart
        self.view = view
        self.predIdx = self.body_parts.index(self.predPart)
        #print("Part idx: ", self.predIdx, '\n')
        
        #get model
        if model == 0:
            #best model-TBD
            model = self.data[self.predPart+'_Model'+ str(2)]
        else:
            model = self.data[self.predPart+'_Model'+ str(model)]

        #get view
        #view abnormal flow for all points
        if(self.view == 'all'):
            
            self.accuracy_part = float(accuracy_score(y_true = model.Body_Label,
                                            y_pred = model.Body_Prediction))
            self.truePart = self.predPart
            self.generateNodes(model)
            self.generateLinks(model)
            
        #view abnormal flow for correctly classified points
        if(self.view == 'classified'):
            
            self.truePart = self.predPart
            _model = model[model.Body_Label == self.predIdx]
            self.accuracy_part = 1
            self.generateNodes(_model)
            self.generateLinks(_model)
       
        #view flow of all misclassififed points
        if(self.view == 'misclassified'):
            if(self.truePart == 'all'):
                _model = model[model.Body_Label != self.predIdx]
                self.generateNodes(_model)
                self.generateLinks(_model)
                
            else:
                try:
                    self.trueIdx = self.body_parts.index(self.truePart)
                    #print("true part idx: ", self.trueIdx)
                except:
                    msg = "incorrect body part name"
                    print(msg)
                _model = model[model.Body_Label == self.trueIdx]
                self.accuracy_part = 0
                self.generateNodes(_model)
                self.generateLinks(_model)
        
        dNetJson = {'nodes': self.nodes, 'links': self.links, 'confusion_matrix': self.cm}
        
        return dNetJson
    
    def generateNodes(self, model):
        
        fp = 0
        fn = 0
        
        if(model.shape[0] == 1):
            tp = 1 if(model.Abnormal_Label.iloc[0] == 1 & model.Abnormal_Prediction.iloc[0] == 1) else 0
            tn = 1 if(model.Abnormal_Label.iloc[0] == 0 & model.Abnormal_Prediction.iloc[0] == 0) else 0
            
        else:
            tn, fp, fn, tp = confusion_matrix(y_true = model.Abnormal_Label, 
                                              y_pred = model.Abnormal_Prediction).ravel()
        self.cm = { self.ckeys[0] : int(tn),
               self.ckeys[1] : int(fp),
               self.ckeys[2] : int(fn),
               self.ckeys[3] : int(tp) }
        
        if(self.truePart == 'all'):
            misclass_counts = model.Body_Label.value_counts(normalize = True)
            print(misclass_counts)
            other = [{self.nkeys[0] :  self.body_parts[idx],
                      'h' : float(misclass_counts[idx]),
                      self.nkeys[1] : self.color[self.body_parts[idx]]} 
                       for idx in misclass_counts.keys()]
            self.nodes = [{self.nkeys[0] : 'Other', 
                      'parts' : other}]
        else: 
            #set node dictionary  
            self.nodes = [{self.nkeys[0] : self.truePart,                              #name
                      self.nkeys[1] : self.color[self.truePart],                       #color
                      self.nkeys[2] : self.accuracy_part}]                             #accuracy
        
        try:
            accNormal = float(tp/(tp+fn))
            accAbnormal = float(tn/(tn+fp))
        except:
            accNormal = tp
            accAbnormal = tn

        self.nodes.extend([
            {self.nkeys[0] : self.classes[0], 
             self.nkeys[1] : self.color[self.classes[0]], 
             self.nkeys[2] : float(tp) if math.isnan(accNormal) else accNormal},
             
            {self.nkeys[0] : self.classes[1],
             self.nkeys[1] : self.color[self.classes[1]],
             self.nkeys[2] : float(tn) if math.isnan(accAbnormal) else accAbnormal}])
        
            
    def generateLinks(self, model):
        
        # gives a pandas series object; get counts of a body-part classified as normal/abnormal
        # by indexing [body_label, abnormal_label]; 
        part_abnormality = model.groupby(['Body_Label', 'Abnormal_Prediction']).size()
        filter_parts = part_abnormality.index.get_level_values(0)                       #body_label
        filter_abnormality = part_abnormality.index.get_level_values(1)                 #abnormality
        for part, abn in zip(filter_parts, filter_abnormality):
            self.links.extend([{ self.lkeys[0] : 0,                                          #source
                     self.lkeys[1] : int(abn)+1,                                        #tagert
                     self.lkeys[2] : 1,                                                 #value
                     self.lkeys[3] : int(part_abnormality[part, abn]),                  #count
                     self.lkeys[4] : self.color[self.body_parts[part]]}])                   #color

In [89]:
c = DenseNetViz(kwargs = {'data' : data,                                   
                'body_parts' : body_parts,
                'color' : color})

In [90]:
confusion_matrix([0],[0]).ravel()

array([1])

In [91]:
c.get('Shoulder', model = 1, view = 'misclassified',truePart = 'Finger')

{'nodes': [{'name': 'Finger', 'color': '#fee090', 'accuracy': 0},
  {'name': 'Normal', 'color': '#5ab4ac', 'accuracy': 0},
  {'name': 'Abnormal', 'color': '#d8b365', 'accuracy': 1}],
 'links': [{'source': 0,
   'target': 1,
   'value': 1,
   'count': 1,
   'color': '#fee090'}],
 'confusion_matrix': {'tn': 1, 'fp': 0, 'fn': 0, 'tp': 0}}